# All-probe Bombcell dashboard (ROI-aware)

This notebook loads **all probes simultaneously** and builds ROI-aware tables/plots.

## How to use `IN_ROI` / `OUTSIDE_ROI`
1. JSON defaults are read from `probe_recording_roi` in your Grant config file.
2. You can override any probe ROI in-notebook using `ROI_OVERRIDE_BY_PROBE`.
3. Units with `distance_from_tip_um <= ROI_END_UM_BY_PROBE[probe]` are labeled `IN_ROI`.
4. Units above that cutoff are labeled `OUTSIDE_ROI`.
5. If a probe has no ROI value (`None`), units are labeled `ROI_NOT_SET`.


In [ ]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bombcell as bc

analysis_dir = Path.cwd().resolve()
sys.path.insert(0, str(analysis_dir))
from post_analysis_setup import load_post_analysis_context, label_units_by_tip_distance

CONFIG_FILE = '../configs/grant_recording_config.json'
RUN_MODE = 'batch'  # batch | np20_rerun | single_probe

# Optional notebook-only ROI overrides (um from tip)
ROI_OVERRIDE_BY_PROBE = {
    # 'B': 950,
}
TIP_POSITION_BY_PROBE = {
    'A': 'min_y', 'B': 'min_y', 'C': 'min_y',
    'D': 'min_y', 'E': 'min_y', 'F': 'min_y',
}


In [ ]:
ctx = load_post_analysis_context(CONFIG_FILE)
mode_to_roots = {
    'batch': ctx['DEFAULT_KS_STAGING_ROOT'],
    'np20_rerun': ctx['NP20_KS_STAGING_ROOT'],
    'single_probe': ctx['BOMBCELL_KS_SINGLEPROBE_STAGING_ROOT'],
}
staging_root = mode_to_roots[RUN_MODE]
probe_letters = list(ctx['probeLetters'])

# ROI defaults from JSON config, with notebook overrides layered on top
ROI_END_UM_BY_PROBE = dict(ctx.get('PROBE_RECORDING_ROI', {}))
ROI_END_UM_BY_PROBE.update(ROI_OVERRIDE_BY_PROBE)

print('staging_root:', staging_root)
print('probes:', probe_letters)
print('ROI defaults/overrides:', ROI_END_UM_BY_PROBE)


## Load all probes and attach ROI labels + classification + reason flags


In [ ]:
all_units = []
missing_probes = []

for probe in probe_letters:
    ks_dir = Path(staging_root) / f'kilosort4_{probe}'
    save_path = ks_dir / 'bombcell'
    if not save_path.exists():
        missing_probes.append(probe)
        continue

    param, quality_metrics, _ = bc.load_bc_results(str(save_path))
    unit_type, unit_type_string = bc.qm.get_quality_unit_type(param, quality_metrics)

    qm_df = pd.DataFrame(quality_metrics).copy()
    qm_df['probe'] = probe
    qm_df['ks_dir'] = str(ks_dir)
    qm_df['save_path'] = str(save_path)
    qm_df['bombcell_label'] = unit_type_string
    qm_df['unit_index'] = np.arange(len(qm_df))

    if 'cluster_id' not in qm_df.columns:
        qm_df['cluster_id'] = qm_df['unit_index']

    roi_end = ROI_END_UM_BY_PROBE.get(probe, None)
    if roi_end is None:
        qm_df['distance_from_tip_um'] = np.nan
        qm_df['roi_label'] = 'ROI_NOT_SET'
    else:
        qm_df = label_units_by_tip_distance(
            quality_metrics=qm_df,
            ks_dir=ks_dir,
            roi_end_um=float(roi_end),
            tip_position=TIP_POSITION_BY_PROBE.get(probe, 'min_y'),
            in_label='IN_ROI',
            out_label='OUTSIDE_ROI',
        )

    # Build reason flags from Bombcell qm table (boolean columns)
    qm_table = bc.make_qm_table(quality_metrics, param, unit_type_string)
    bool_cols = [c for c in qm_table.columns if qm_table[c].dtype == bool]
    reason_tokens = []
    for _, row in qm_table[bool_cols].iterrows():
        hits = [c for c in bool_cols if bool(row[c])]
        reason_tokens.append(hits)

    qm_df['reason_tokens'] = reason_tokens
    qm_df['n_reason_flags'] = qm_df['reason_tokens'].apply(len)

    all_units.append(qm_df)

if len(all_units) == 0:
    raise RuntimeError('No probe outputs were loaded. Check RUN_MODE/staging root and that bombcell outputs exist.')

all_df = pd.concat(all_units, ignore_index=True)
print('Loaded units:', len(all_df))
print('Missing probes (no save_path):', missing_probes)
print(all_df[['probe','bombcell_label','roi_label']].head())


## Unified all-probe dataframe views


In [ ]:
summary_probe_roi_class = (
    all_df.groupby(['probe', 'roi_label', 'bombcell_label'])
    .size()
    .rename('n_units')
    .reset_index()
)
display(summary_probe_roi_class.sort_values(['probe', 'roi_label', 'bombcell_label']).head(200))

pivot_probe_class = all_df.pivot_table(index='probe', columns='bombcell_label', values='cluster_id', aggfunc='count', fill_value=0)
display(pivot_probe_class)

pivot_probe_roi = all_df.pivot_table(index='probe', columns='roi_label', values='cluster_id', aggfunc='count', fill_value=0)
display(pivot_probe_roi)


## ROI-aware plots across all probes


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1) Units per probe split by ROI label
probe_roi = all_df.pivot_table(index='probe', columns='roi_label', values='cluster_id', aggfunc='count', fill_value=0)
probe_roi.plot(kind='bar', stacked=True, ax=axes[0,0])
axes[0,0].set_title('Units per probe (ROI split)')
axes[0,0].set_ylabel('n units')
axes[0,0].tick_params(axis='x', rotation=0)

# 2) Units per class split by ROI label
class_roi = all_df.pivot_table(index='bombcell_label', columns='roi_label', values='cluster_id', aggfunc='count', fill_value=0)
class_roi.plot(kind='bar', stacked=True, ax=axes[0,1])
axes[0,1].set_title('Bombcell classes (ROI split)')
axes[0,1].set_ylabel('n units')
axes[0,1].tick_params(axis='x', rotation=35)

# 3) Mean reason flags per class and ROI
mean_flags = all_df.groupby(['bombcell_label','roi_label'])['n_reason_flags'].mean().unstack(fill_value=0)
mean_flags.plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Mean # reason flags per unit (class × ROI)')
axes[1,0].set_ylabel('mean n_reason_flags')
axes[1,0].tick_params(axis='x', rotation=35)

# 4) Probe × class heatmap for IN_ROI only (or fallback ALL)
in_roi = all_df[all_df['roi_label'] == 'IN_ROI']
plot_df = in_roi if len(in_roi) > 0 else all_df
heat = plot_df.pivot_table(index='probe', columns='bombcell_label', values='cluster_id', aggfunc='count', fill_value=0)
im = axes[1,1].imshow(heat.to_numpy(), aspect='auto')
axes[1,1].set_xticks(np.arange(len(heat.columns)))
axes[1,1].set_xticklabels(list(heat.columns), rotation=35, ha='right')
axes[1,1].set_yticks(np.arange(len(heat.index)))
axes[1,1].set_yticklabels(list(heat.index))
axes[1,1].set_title('Probe × class heatmap (' + ('IN_ROI' if len(in_roi)>0 else 'ALL units') + ')')
plt.colorbar(im, ax=axes[1,1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


## Reason-level plots with ROI as a key dimension


In [ ]:
from collections import Counter

# explode reasons for long-form reason table
reason_rows = []
for _, row in all_df.iterrows():
    for r in row['reason_tokens']:
        reason_rows.append({'probe': row['probe'], 'roi_label': row['roi_label'], 'bombcell_label': row['bombcell_label'], 'reason': r})
reason_df = pd.DataFrame(reason_rows)

if len(reason_df) == 0:
    print('No reason flags available in reason_df')
else:
    # top reasons overall
    top_reasons = reason_df['reason'].value_counts().head(12).index
    rr = reason_df[reason_df['reason'].isin(top_reasons)]

    fig, axes = plt.subplots(1, 3, figsize=(23, 6))

    # A) top reasons by ROI
    a = rr.groupby(['reason','roi_label']).size().unstack(fill_value=0).loc[top_reasons]
    a.plot(kind='barh', stacked=True, ax=axes[0])
    axes[0].set_title('Top reason counts (ROI split)')
    axes[0].set_xlabel('count')

    # B) top reasons by probe (stacked)
    b = rr.groupby(['reason','probe']).size().unstack(fill_value=0).loc[top_reasons]
    b.plot(kind='bar', stacked=True, ax=axes[1])
    axes[1].set_title('Top reason counts by probe')
    axes[1].tick_params(axis='x', rotation=60)

    # C) reason heatmap for IN_ROI (or OUTSIDE fallback)
    focus = rr[rr['roi_label']=='IN_ROI']
    if len(focus)==0:
        focus = rr[rr['roi_label']=='OUTSIDE_ROI']
    h = focus.pivot_table(index='reason', columns='probe', values='bombcell_label', aggfunc='count', fill_value=0)
    h = h.loc[h.sum(axis=1).sort_values(ascending=False).head(12).index]
    im = axes[2].imshow(h.to_numpy(), aspect='auto')
    axes[2].set_yticks(np.arange(len(h.index)))
    axes[2].set_yticklabels(list(h.index))
    axes[2].set_xticks(np.arange(len(h.columns)))
    axes[2].set_xticklabels(list(h.columns))
    axes[2].set_title('Reason heatmap in ROI focus set')
    plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()


## Individual-unit + probe-level drill-down (ROI-aware)


In [ ]:
# Pick any probe and class; then inspect units + reason flags
DRILL_PROBE = 'B'
DRILL_CLASS = None  # e.g. 'MUA', 'GOOD', 'NOISE', 'NON-SOMA'; None keeps all
DRILL_ROI = 'IN_ROI'  # IN_ROI | OUTSIDE_ROI | ROI_NOT_SET | None

sub = all_df[all_df['probe'] == DRILL_PROBE].copy()
if DRILL_CLASS is not None:
    sub = sub[sub['bombcell_label'].astype(str).str.contains(DRILL_CLASS, regex=False)]
if DRILL_ROI is not None:
    sub = sub[sub['roi_label'] == DRILL_ROI]

print('n units in drill subset:', len(sub))
display(sub[['probe','cluster_id','bombcell_label','roi_label','distance_from_tip_um','n_reason_flags']].head(100))


In [ ]:
if len(sub) == 0:
    print('Drill subset empty. Adjust DRILL_* filters.')
else:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # distance distribution by class within drill subset
    if sub['distance_from_tip_um'].notna().any():
        for label, grp in sub.groupby('bombcell_label'):
            vals = grp['distance_from_tip_um'].dropna().to_numpy()
            if len(vals) > 0:
                axes[0].hist(vals, bins=30, alpha=0.5, label=str(label))
        axes[0].set_title(f'Distance-from-tip distribution ({DRILL_PROBE}, ROI={DRILL_ROI})')
        axes[0].set_xlabel('distance_from_tip_um')
        axes[0].set_ylabel('n units')
        axes[0].legend(fontsize=8)
    else:
        axes[0].text(0.5,0.5,'No distance data for this subset',ha='center',va='center')

    # top reasons in drill subset
    from collections import Counter
    c = Counter(r for row in sub['reason_tokens'] for r in row)
    if c:
        top = c.most_common(15)
        labs = [k for k,_ in top][::-1]
        vals = [v for _,v in top][::-1]
        axes[1].barh(labs, vals)
    else:
        axes[1].text(0.5,0.5,'No reason flags in subset',ha='center',va='center')
    axes[1].set_title(f'Top reasons ({DRILL_PROBE}, ROI={DRILL_ROI})')
    axes[1].set_xlabel('count')

    plt.tight_layout()
    plt.show()


## Optional export


In [ ]:
out_csv = Path(staging_root) / f'all_probes_{RUN_MODE}_roi_audit.csv'
all_df.to_csv(out_csv, index=False)
print('Saved all-probe ROI-aware dataframe to:', out_csv)
